# Building an Agentic RAG System with Open-Source LLMs

This notebook demonstrates how to build an advanced Retrieval-Augmented Generation (RAG) system to answer complex questions from a long document. Instead of relying on a traditional vector database, we will employ a multi-agent, hierarchical approach where LLM-powered agents navigate the document's structure dynamically.

This method is particularly useful for dense, lengthy texts like legal manuals, research papers, or financial reports, where context is spread across many pages. Our system will mimic how a human researcher would work: first skimming for relevant sections, then drilling down into specific paragraphs, and finally synthesizing an answer based only on the retrieved information.

**Key Features of this Approach:**

1.  **Zero-Ingestion:** The system can work with new documents instantly without any pre-processing or embedding steps.
2.  **Dynamic Retrieval:** The LLM itself decides which parts of the document are relevant, allowing it to handle paraphrased or conceptual questions more effectively.
3.  **Traceability:** The system provides precise, paragraph-level citations for every part of its answer, ensuring verifiability.
4.  **Customizable:** The entire workflow is built to be compatible with open-source LLMs accessible through OpenAI-compatible APIs.

## 1. Setup and Configuration

First, we'll install the necessary libraries and set up our environment. This includes libraries for handling PDFs, interacting with the LLM API, and basic data manipulation.

In [1]:
# %pip install -qU openai requests pypdf nltk transformers pandas tqdm

### 1.1. Import Libraries and Configure LLM Client

Next, we'll import all the required modules. We will also configure our LLM client here. This notebook is designed to work with any LLM that provides an OpenAI-compatible API endpoint.

**Important:** You must provide your own `API_KEY` and `BASE_URL` in the cell below. We also define different model names for each task (routing, synthesis, verification) to allow for using specialized or cost-effective models at different stages.

In [2]:
import os
import json
import re
import time
import requests
import pandas as pd
from io import BytesIO
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pypdf import PdfReader
from transformers import AutoTokenizer
from typing import List, Dict, Any
import nltk
from nltk.tokenize import sent_tokenize
from tqdm.auto import tqdm



# Choose a local model you can actually load (8B is OK for A40 with 46GB)
ROUTER_MODEL = "meta-llama/Llama-3.1-8B"
SYNTHESIS_MODEL = "meta-llama/Llama-3.1-8B"  # keep same to simplify
EVALUATION_MODEL = "meta-llama/Llama-3.1-8B"

tokenizer = AutoTokenizer.from_pretrained(ROUTER_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    ROUTER_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

# --- Global Variables ---
# We will store performance metrics here throughout the run
metrics_log = []

# Download necessary NLTK data for sentence tokenization
nltk.download('punkt_tab', quiet=True)

`torch_dtype` is deprecated! Use `dtype` instead!
/u/ilacp2/.conda/envs/llm/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

True

In [13]:
def local_llm_generate(prompt, model, tokenizer, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


### 1.2. Initialize Tokenizer for Estimation

The primary purpose of a local tokenizer in this pipeline is to **estimate token counts**. This is crucial for two reasons: 1) to avoid sending requests that exceed the model's context window limit, which would cause an API error, and 2) to estimate the cost of an API call before making it.

Since the Router Agent receives the largest prompts (containing many document chunks), its context window is the most likely to be exceeded. Therefore, our local tokenizer should be the best possible proxy for the tokenizer used by the `ROUTER_MODEL`.

In [4]:
# Use the tokenizer from our designated ROUTER_MODEL for consistent token counting.
print(f"Initializing tokenizer for '{ROUTER_MODEL}'...")
tokenizer = AutoTokenizer.from_pretrained(ROUTER_MODEL)

def count_tokens(text: str) -> int:
    """Estimates the number of tokens in a string using the reference tokenizer."""
    if not isinstance(text, str):
        return 0
    return len(tokenizer.encode(text))

Initializing tokenizer for 'meta-llama/Llama-3.1-8B'...


## 2. Document Loading and Preparation

Our process begins by loading the source document. For this example, we'll use the *Trademark Trial and Appeal Board Manual of Procedure (TBMP)*, a lengthy legal document that serves as a great test case. We'll download the PDF, extract its text content, and analyze its size.

In [5]:
def load_pdf_from_url(url: str, max_pages: int = 920) -> str:
    """
    Downloads a PDF from a URL, extracts text from its pages, and returns it as a single string.
    """
    print(f"Downloading document from {url}...")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Ensure the download was successful
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        return ""

    pdf_file = BytesIO(response.content)
    pdf_reader = PdfReader(pdf_file)
    
    num_pages_to_process = min(max_pages, len(pdf_reader.pages))
    print(f"Extracting text from {num_pages_to_process} pages...")
    
    full_text = ""
    # Use tqdm for a progress bar during page extraction
    for page in tqdm(pdf_reader.pages[:num_pages_to_process], desc="Extracting pages"):
        page_text = page.extract_text()
        if page_text:
            full_text += page_text + "\n"
    
    return full_text

In [6]:
# URL for the TBMP manual
tbmp_url = "https://www.uspto.gov/sites/default/files/documents/tbmp-Master-June2024.pdf"
document_text = load_pdf_from_url(tbmp_url)

# Display document statistics
char_count = len(document_text)
token_count = count_tokens(document_text)
print(f"\nDocument loaded successfully.")
print(f"- Total Characters: {char_count:,}")
print(f"- Estimated Tokens: {token_count:,}")

print("\n--- Document Preview (first 500 characters) ---")
print(document_text[:500])
print("---------------------------------------------")

Extracting text from 920 pages...


Extracting pages:   0%|          | 0/920 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (945084 > 131072). Running this sequence through the model will result in indexing errors



Document loaded successfully.
- Total Characters: 3,459,491
- Estimated Tokens: 945,084

--- Document Preview (first 500 characters) ---
TRADEMARK TRIAL AND
APPEAL BOARD MANUAL
OF PROCEDURE (TBMP)
 June 2024
June   2024
United States Patent and Trademark Office
PREFACE TO THE JUNE 2024 REVISION
The June 2024 revision of the Trademark Trial and Appeal Board Manual of Procedure is an update of the
June 2023 edition. This update is moderate in nature and incorporates relevant case law issued between March
3, 2023 and March 1, 2024.
The title of the manual is abbreviated as “TBMP.” A citation to a section of the manual may be written
---------------------------------------------


## 3. Hierarchical Chunking

The document is far too large to fit into a single model context. Instead of creating hundreds of small, independent chunks for a vector database, we will create a small number of large, high-level chunks (e.g., 20). This forms the top level of our document hierarchy.

Our chunking function is designed to be "sentence-aware," meaning it tries to avoid splitting sentences in the middle, which helps preserve the semantic integrity of the text.

In [7]:
def split_text_into_chunks(text: str, num_chunks: int = 20) -> List[Dict[str, Any]]:
    """
    Splits a long text into a specified number of chunks, respecting sentence boundaries.
    """
    # First, split the entire text into individual sentences
    sentences = sent_tokenize(text)
    if not sentences:
        return []

    # Calculate how many sentences should go into each chunk on average
    sentences_per_chunk = (len(sentences) + num_chunks - 1) // num_chunks

    chunks = []
    desc = "Creating chunks" if len(sentences) > 500 else None # Only show progress bar if it's a long process
    for i in tqdm(range(0, len(sentences), sentences_per_chunk), desc=desc):
        chunk_sentences = sentences[i:i + sentences_per_chunk]
        chunk_text = " ".join(chunk_sentences)
        chunks.append({
            "id": len(chunks),  # Assign a simple integer ID
            "text": chunk_text
        })
    
    print(f"Document split into {len(chunks)} chunks.")
    return chunks

In [8]:
document_chunks = split_text_into_chunks(document_text, num_chunks=20)

# Display stats for the first few chunks
for chunk in document_chunks[:3]:
    chunk_token_count = count_tokens(chunk['text'])
    print(f"- Chunk {chunk['id']}: {chunk_token_count:,} tokens")

Creating chunks:   0%|          | 0/20 [00:00<?, ?it/s]

Document split into 20 chunks.
- Chunk 0: 42,822 tokens
- Chunk 1: 42,367 tokens
- Chunk 2: 42,516 tokens


## 4. The Agentic Navigation Workflow

This is the core of our system. We will create a recursive process that navigates the document hierarchy to find the most relevant paragraphs for a given question. The workflow consists of two main components:

1.  **Router Agent:** An LLM that examines a set of chunks and selects which ones are relevant. We implement this as a two-step process: first, the agent writes its reasoning to a "scratchpad," and second, it makes its final selection. This separation improves the quality of its decisions.
2.  **Recursive Navigator:** A loop that repeatedly calls the Router Agent. It starts with top-level chunks. When the agent selects chunks, the navigator splits them into smaller sub-chunks and presents those to the agent in the next step. This continues until a maximum depth is reached.

### 4.1. Helper Function for Robust JSON Parsing

Since we will be asking the LLM to return JSON formatted text, we need a reliable way to parse it. Models sometimes wrap their JSON output in markdown code blocks (e.g., ` ```json ... ``` `) or add explanatory text. This helper function is designed to find and parse the JSON block, making our system more resilient to formatting variations.

In [9]:
def parse_json_from_response(text: str) -> Dict[str, Any]:
    """
    Extracts and parses a JSON object from a string, even if it's embedded in markdown.
    """
    match = re.search(r'```(?:json)?\s*({.*?})\s*```', text, re.S)
    if match:
        json_str = match.group(1)
    else:
        start = text.find('{')
        end = text.rfind('}')
        if start != -1 and end != -1:
            json_str = text[start:end+1]
        else:
            json_str = text
    
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        print(f"Warning: Failed to parse JSON from response. Raw text: '{text}'")
        return {}

### 4.2. The Router Agent (Two-Pass Logic)

This function encapsulates the two-pass routing logic. It first calls the LLM to generate reasoning, then incorporates that reasoning into a second call to make the final selection. This emulates a more deliberate thought process and leads to better results.

In [10]:
def route_to_chunks(question: str, chunks: List[Dict[str, Any]], scratchpad: str, depth: int) -> Dict[str, Any]:
    """
    Uses a two-pass LLM approach to select relevant chunks for a given question.
    """
    print(f"\n--- Routing at Depth {depth}: Evaluating {len(chunks)} chunks ---")
    
    chunks_formatted = "\n\n".join([f"CHUNK {chunk['id']}:\n{chunk['text'][:1000]}..." for chunk in chunks])
    reasoning_prompt = f"""
    You are an expert document analyst. Your goal is to find information to answer the user's question:
    '{question}'
    
    Here is your reasoning so far:
    {scratchpad}
    
    Review the following new text chunks. Briefly explain which chunks seem relevant to the question and why. This is your internal monologue.
    
    TEXT CHUNKS:
    {chunks_formatted}
    
    Your Reasoning:
    """
    
    start_time = time.time()
    reasoning_response = local_llm_generate(reasoning_prompt, model, tokenizer)
    latency_1 = time.time() - start_time
    
    new_reasoning = reasoning_response.choices[0].message.content
    updated_scratchpad = scratchpad + f"\n[Depth {depth} Reasoning]: {new_reasoning}"
    print(f"LLM Reasoning: {new_reasoning}")
    
    p_tokens_1, c_tokens_1 = reasoning_response.usage.prompt_tokens, reasoning_response.usage.completion_tokens
    metrics_log.append({"step": f"route_depth_{depth}_reason", "model": ROUTER_MODEL, "latency_s": latency_1, "prompt_tokens": p_tokens_1, "completion_tokens": c_tokens_1, "total_tokens": p_tokens_1 + c_tokens_1})

    selection_prompt = f"""
    Based on your reasoning below, select the chunk IDs that are most likely to contain the answer to the question: '{question}'.
    
    Your Reasoning:
    {new_reasoning}
    
    TEXT CHUNKS:
    {chunks_formatted}
    
    Respond with ONLY a valid JSON object with a single key 'selected_chunk_ids', which is a list of integers. Example: {{"selected_chunk_ids": [1, 5, 8]}}
    """
    
    start_time = time.time()
    selection_response = local_llm_generate(reasoning_prompt, model, tokenizer)

    latency_2 = time.time() - start_time

    response_text = selection_response.choices[0].message.content
    parsed_output = parse_json_from_response(response_text)
    selected_ids = parsed_output.get('selected_chunk_ids', [])
    print(f"Selected chunk IDs: {selected_ids}")
    
    p_tokens_2, c_tokens_2 = selection_response.usage.prompt_tokens, selection_response.usage.completion_tokens
    metrics_log.append({"step": f"route_depth_{depth}_select", "model": ROUTER_MODEL, "latency_s": latency_2, "prompt_tokens": p_tokens_2, "completion_tokens": c_tokens_2, "total_tokens": p_tokens_2 + c_tokens_2})

    return {"selected_ids": selected_ids, "scratchpad": updated_scratchpad}

### 4.3. The Recursive Navigator

This function orchestrates the entire navigation process. It initializes the loop with the top-level document chunks and calls the router. For each selected chunk, it splits it into smaller sub-chunks and continues the process until the `max_depth` is reached. The path of each chunk (e.g., `3.5.2`) is tracked to provide clear, hierarchical citations in the final output.

In [11]:
def navigate_document(question: str, initial_chunks: List[Dict[str, Any]], max_depth: int = 2) -> Dict[str, Any]:
    """
    Performs a hierarchical navigation of the document to find relevant paragraphs.
    """
    scratchpad = ""
    current_chunks = initial_chunks
    final_paragraphs = []
    
    chunk_paths = {chunk["id"]: str(chunk["id"]) for chunk in initial_chunks}

    for depth in tqdm(range(max_depth), desc="Navigating Document"):
        result = route_to_chunks(question, current_chunks, scratchpad, depth)
        scratchpad = result["scratchpad"]
        selected_ids = result["selected_ids"]
        
        if not selected_ids:
            print("\nNavigation stopped: No relevant chunks selected.")
            final_paragraphs = current_chunks
            break

        selected_chunks = [c for c in current_chunks if c["id"] in selected_ids]

        next_level_chunks = []
        next_chunk_id_counter = 0
        for chunk in selected_chunks:
            parent_path = chunk_paths[chunk["id"]]
            sub_chunks = split_text_into_chunks(chunk['text'], num_chunks=10)
            
            for i, sub_chunk in enumerate(sub_chunks):
                new_id = next_chunk_id_counter
                sub_chunk['id'] = new_id
                chunk_paths[new_id] = f"{parent_path}.{i}"
                next_level_chunks.append(sub_chunk)
                next_chunk_id_counter += 1
        
        current_chunks = next_level_chunks
        final_paragraphs = current_chunks
        
    print(f"\nNavigation finished. Returning {len(final_paragraphs)} retrieved paragraphs.")
    for chunk in final_paragraphs:
        if chunk['id'] in chunk_paths:
             chunk['display_id'] = chunk_paths[chunk['id']]
        
    return {"paragraphs": final_paragraphs, "scratchpad": scratchpad}

### 4.4. Run the Full Navigation Process

Now we'll execute the navigation with a sample question. This will perform multiple LLM calls and drill down into the document to find the most relevant information.

In [14]:
sample_question = "What are the requirements for filing a motion to compel discovery, including formatting and signatures?"

metrics_log = [] 

navigation_result = navigate_document(sample_question, document_chunks, max_depth=2)

print(f"\n--- Navigation Complete ---")
print(f"Retrieved {len(navigation_result['paragraphs'])} paragraphs for synthesis.")

if navigation_result['paragraphs']:
    first_para = navigation_result['paragraphs'][0]
    print(f"\n--- Preview of Retrieved Paragraph {first_para.get('display_id', 'N/A')} ---")
    print(first_para['text'][:500] + "...")
    print("---------------------------------------")

Navigating Document:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Routing at Depth 0: Evaluating 20 chunks ---


KeyboardInterrupt: 

## 5. Answer Synthesis

After the navigation phase, we have a curated set of paragraphs that are highly relevant to the question. The next step is to use a **Synthesizer Agent** to generate a comprehensive, human-readable answer based *only* on this retrieved context.

We will instruct the model to cite the `display_id` of the paragraphs it uses, ensuring every piece of information in the answer is traceable back to its source.

In [ ]:
def generate_answer(question: str, paragraphs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Generates a final answer based on the retrieved paragraphs.
    """
    print("\n--- Synthesizing Final Answer ---")
    
    if not paragraphs:
        return {"answer": "I could not find relevant information to answer the question.", "citations": []}

    context = "\n\n".join([f"PARAGRAPH {p.get('display_id', p['id'])}:\n{p['text']}" for p in paragraphs])
    
    system_prompt = """
    You are a legal research assistant. Your task is to answer the user's question based *only* on the provided paragraphs from a legal manual.
    - Synthesize the information from the paragraphs into a clear and concise answer.
    - For every statement you make, you MUST cite the paragraph ID(s) it is based on in parentheses, like (ID: 1.2.5).
    - If the provided paragraphs do not contain enough information, state that clearly.
    - Do not use any external knowledge.
    - Respond with a JSON object containing 'answer' and 'citations' (a list of all unique IDs you cited).
    """
    
    user_prompt = f"""
    USER QUESTION: "{question}"
    
    SOURCE PARAGRAPHS:
    {context}
    
    Please provide your answer in the required JSON format.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    start_time = time.time()
    response = local_llm_generate(system_prompt, model, tokenizer)
    latency = time.time() - start_time

    response_text = response.choices[0].message.content
    p_tokens, c_tokens = response.usage.prompt_tokens, response.usage.completion_tokens
    metrics_log.append({"step": "synthesis", "model": SYNTHESIS_MODEL, "latency_s": latency, "prompt_tokens": p_tokens, "completion_tokens": c_tokens, "total_tokens": p_tokens + c_tokens})
    
    parsed_output = parse_json_from_response(response_text)
    return {
        "answer": parsed_output.get("answer", "Failed to generate a valid answer."),
        "citations": sorted(list(set(parsed_output.get("citations", []))))
    }

In [13]:
final_answer_result = generate_answer(
    sample_question, 
    navigation_result['paragraphs']
)

print("\n--- GENERATED ANSWER ---")
print(final_answer_result['answer'])
print("\n--- CITATIONS ---")
print(final_answer_result['citations'])


--- Synthesizing Final Answer ---

--- GENERATED ANSWER ---
The requirements for filing a motion to compel discovery include filing the motion before the deadline for pretrial disclosures for the first testimony period, as originally set or as reset (9.0.1, 9.0.5). The motion must be filed with the Board and must include a certification that the movant has in good faith conferred or attempted to confer with other affected parties in an effort to resolve the dispute without court action (9.0.8.0). The party seeking discovery must demonstrate that the responding party has failed to answer any interrogatory or produce and permit the inspection and copying of any document or thing (9.0.1). The motion to compel must be in writing and must specify the discovery requests that the responding party has failed to answer or produce (9.0.1). The Board may grant the motion to compel if it determines that the responding party has failed to comply with the discovery requests and that the requesting 

## 6. Qualitative Evaluation (LLM-as-Judge)

To ensure the quality and trustworthiness of our system, we add a final evaluation stage with multiple checks. We use a powerful **Evaluation Agent** to act as a judge on several criteria.

1.  **Faithfulness:** Checks if the answer is factually consistent with the cited source paragraphs.
2.  **Answer Relevance:** Scores how well the answer addresses the original question.
3.  **Retrieval Relevance:** Scores how relevant the retrieved paragraphs were for answering the question.

In [ ]:
def evaluate_faithfulness(question: str, answer: str, citations: List[str], paragraphs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Uses an LLM to verify if the answer is fully supported by the cited paragraphs.
    """
    print("\n--- Evaluating Answer Faithfulness ---")
    
    if not citations or not answer:
        return {"is_faithful": False, "explanation": "No answer or citations provided."}
        
    cited_paragraphs = [p for p in paragraphs if p.get('display_id') in citations]
    if not cited_paragraphs:
        return {"is_faithful": False, "explanation": f"Cited IDs {citations} not found."}
        
    context = "\n\n".join([f"PARAGRAPH {p['display_id']}:\n{p['text']}" for p in cited_paragraphs])
    
    prompt = f"""
    You are a meticulous fact-checker. Determine if the 'ANSWER' is fully supported by the 'SOURCE PARAGRAPHS'.
    The answer is 'faithful' only if every single piece of information it contains is directly stated or logically derived from the source paragraphs.
    
    QUESTION: "{question}"
    ANSWER TO VERIFY: "{answer}"
    SOURCE PARAGRAPHS:
    {context}
    
    Respond with a JSON object: {{"is_faithful": boolean, "explanation": "brief reasoning"}}.
    """
    
    start_time = time.time()
    response = local_llm_generate(prompt, model, tokenizer)

    latency = time.time() - start_time

    response_text = response.choices[0].message.content
    p_tokens, c_tokens = response.usage.prompt_tokens, response.usage.completion_tokens
    metrics_log.append({"step": "eval_faithfulness", "model": EVALUATION_MODEL, "latency_s": latency, "prompt_tokens": p_tokens, "completion_tokens": c_tokens, "total_tokens": p_tokens + c_tokens})

    return parse_json_from_response(response_text)

In [ ]:
def evaluate_answer_relevance(question: str, answer: str) -> Dict[str, Any]:
    """
    Scores the relevance of the generated answer to the original question.
    """
    print("\n--- Evaluating Answer Relevance ---")
    prompt = f"""
    Score how well the 'ANSWER' addresses the 'ORIGINAL QUESTION' on a scale from 0.0 to 1.0.
    - A score of 1.0 means the answer completely and directly answers the question.
    - A score of 0.0 means the answer is completely irrelevant.
    
    ORIGINAL QUESTION: "{question}"
    ANSWER: "{answer}"
    
    Respond with a JSON object: {{"score": float, "justification": "brief reasoning"}}.
    """
    
    start_time = time.time()
    response = local_llm_generate(prompt, model, tokenizer)

    latency = time.time() - start_time
    
    response_text = response.choices[0].message.content
    p_tokens, c_tokens = response.usage.prompt_tokens, response.usage.completion_tokens
    metrics_log.append({"step": "eval_answer_relevance", "model": EVALUATION_MODEL, "latency_s": latency, "prompt_tokens": p_tokens, "completion_tokens": c_tokens, "total_tokens": p_tokens + c_tokens})
    
    parsed = parse_json_from_response(response_text)
    return {"score": parsed.get("score", 0.0), "justification": parsed.get("justification", "")}

In [ ]:
def evaluate_retrieval_relevance(question: str, paragraphs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Scores the relevance of the retrieved documents to the question.
    """
    print("\n--- Evaluating Retrieval Relevance ---")
    context = "\n\n".join([f"PARAGRAPH {p.get('display_id', p['id'])}:\n{p['text'][:500]}..." for p in paragraphs])
    
    prompt = f"""
    Score how relevant the provided 'RETRIEVED PARAGRAPHS' are for answering the 'ORIGINAL QUESTION' on a scale from 0.0 to 1.0.
    - A score of 1.0 means the paragraphs contain all the necessary information.
    - A score of 0.0 means the paragraphs are completely irrelevant.
    
    ORIGINAL QUESTION: "{question}"
    RETRIEVED PARAGRAPHS:
    {context}
    
    Respond with a JSON object: {{"score": float, "justification": "brief reasoning"}}.
    """
    
    start_time = time.time()
    response = local_llm_generate(prompt, model, tokenizer)
    latency = time.time() - start_time

    response_text = response.choices[0].message.content
    p_tokens, c_tokens = response.usage.prompt_tokens, response.usage.completion_tokens
    metrics_log.append({"step": "eval_retrieval_relevance", "model": EVALUATION_MODEL, "latency_s": latency, "prompt_tokens": p_tokens, "completion_tokens": c_tokens, "total_tokens": p_tokens + c_tokens})
    
    parsed = parse_json_from_response(response_text)
    return {"score": parsed.get("score", 0.0), "justification": parsed.get("justification", "")}

In [17]:
# Run all qualitative evaluations
faithfulness_result = evaluate_faithfulness(
    sample_question, 
    final_answer_result['answer'], 
    final_answer_result['citations'],
    navigation_result['paragraphs']
)

answer_relevance_result = evaluate_answer_relevance(
    sample_question,
    final_answer_result['answer']
)

retrieval_relevance_result = evaluate_retrieval_relevance(
    sample_question,
    navigation_result['paragraphs']
)

print("\n--- QUALITATIVE EVALUATION SUMMARY ---")
print(f"Faithfulness Check: {'PASSED' if faithfulness_result.get('is_faithful') else 'FAILED'}")
print(f"  -> Explanation: {faithfulness_result.get('explanation')}")
print(f"Answer Relevance Score: {answer_relevance_result.get('score'):.2f}")
print(f"  -> Justification: {answer_relevance_result.get('justification')}")
print(f"Retrieval Relevance Score: {retrieval_relevance_result.get('score'):.2f}")
print(f"  -> Justification: {retrieval_relevance_result.get('justification')}")


--- Evaluating Answer Faithfulness ---

--- Evaluating Answer Relevance ---

--- Evaluating Retrieval Relevance ---

--- QUALITATIVE EVALUATION SUMMARY ---
Faithfulness Check: FAILED
  -> Explanation: The answer includes information about the Board granting the motion to compel if it determines the responding party has failed to comply and the requesting party has made a good faith effort to resolve the dispute, which is not explicitly stated in the source paragraphs. Additionally, the answer mentions the motion must be in writing and specify the discovery requests, which is not directly supported by the provided source paragraphs.
Answer Relevance Score: 0.80
  -> Justification: The answer provides detailed information on the requirements for filing a motion to compel discovery, including deadlines, certifications, and the necessity of demonstrating non-compliance. However, it does not explicitly mention formatting details or signature requirements, which are part of the original que

## 7. Final Analysis and Summary

Finally, we'll consolidate all our metrics—both operational and qualitative—into two clear summaries. The first DataFrame provides a detailed breakdown of each API call, while the second offers a high-level summary of the entire query's performance and quality.

### 7.1. Define Model Pricing

Define the cost per million tokens for all models used. You should get this information from your LLM provider. The prices below are placeholders.

In [21]:
model_prices_per_million_tokens = {
    "meta-llama/Meta-Llama-3.1-8B-Instruct": {
        "input": 0.02,
        "output": 0.06
    },
    "meta-llama/Llama-3.3-70B-Instruct": {
        "input": 0.13,
        "output": 0.40
    },
    "deepseek-ai/DeepSeek-V3": {
        "input": 0.50,
        "output": 1.50
    }
}

### 7.2. Per-Step Operational Metrics

This first DataFrame shows the detailed operational cost and latency for every individual LLM call made during the process.

In [22]:
if metrics_log:
    df_metrics = pd.DataFrame(metrics_log)

    def calculate_cost(row):
        model_name = row['model']
        prices = model_prices_per_million_tokens.get(model_name, {"input": 0, "output": 0})
        input_cost = (row['prompt_tokens'] / 1_000_000) * prices['input']
        output_cost = (row['completion_tokens'] / 1_000_000) * prices['output']
        return input_cost + output_cost

    df_metrics['cost_usd'] = df_metrics.apply(calculate_cost, axis=1)
    
    print("--- Per-Step Performance and Cost Analysis ---")
    print(df_metrics.to_string())
else:
    print("No metrics were logged.")

--- Per-Step Performance and Cost Analysis ---
                       step                                  model  latency_s  prompt_tokens  completion_tokens  total_tokens  cost_usd
0      route_depth_0_reason  meta-llama/Meta-Llama-3.1-8B-Instruct  14.847801           6139                734          6873  0.000167
1      route_depth_0_select  meta-llama/Meta-Llama-3.1-8B-Instruct   0.879542           6877                 12          6889  0.000138
2      route_depth_1_reason  meta-llama/Meta-Llama-3.1-8B-Instruct   5.627172           3706                330          4036  0.000094
3      route_depth_1_select  meta-llama/Meta-Llama-3.1-8B-Instruct   0.698597           3299                 18          3317  0.000067
4                 synthesis      meta-llama/Llama-3.3-70B-Instruct   9.360613          12301                265         12566  0.001705
5         eval_faithfulness                deepseek-ai/DeepSeek-V3   3.267365           1556                 97          1653  0.000924
6

### 7.3. Final Query Summary

This final summary DataFrame provides a holistic, single-row view of the entire query, combining operational totals with the crucial qualitative scores to assess overall success.

In [25]:
if metrics_log:
    # Calculate totals from the detailed metrics log
    total_latency = df_metrics['latency_s'].sum()
    total_cost = df_metrics['cost_usd'].sum()
    total_tokens = df_metrics['total_tokens'].sum()
    
    # Get qualitative scores
    faithfulness_score = 1.0 if faithfulness_result.get('is_faithful') else 0.0
    answer_relevance_score = answer_relevance_result.get('score', 0.0)
    retrieval_relevance_score = retrieval_relevance_result.get('score', 0.0)
    
    # Calculate a simple overall confidence score
    overall_confidence = faithfulness_score * answer_relevance_score * retrieval_relevance_score

    # Create summary dictionary
    summary_data = {
        'question': [sample_question],
        'total_latency_s': [total_latency],
        'total_cost_usd': [total_cost],
        'total_tokens': [total_tokens],
        'faithfulness_check': ['PASSED' if faithfulness_score == 1.0 else 'FAILED'],
        'answer_relevance_score': [answer_relevance_score],
        'retrieval_relevance_score': [retrieval_relevance_score],
        'overall_confidence_score': [overall_confidence]
    }
    
    df_summary = pd.DataFrame(summary_data)
    
    print("--- Final Query Summary ---")
    # Transpose for better readability of a single-row summary
    print(df_summary.T.rename(columns={0: 'Result'}))
else:
    print("Cannot generate summary as no metrics were logged.")

--- Final Query Summary ---
                                                                      Result
question                   What are the requirements for filing a motion ...
total_latency_s                                                     40.20659
total_cost_usd                                                      0.005851
total_tokens                                                           40568
faithfulness_check                                                    FAILED
answer_relevance_score                                                   0.8
retrieval_relevance_score                                                0.2
overall_confidence_score                                                 0.0


## 8. Conclusion

This notebook has demonstrated an end-to-end agentic RAG workflow using customizable, open-source LLMs. By employing a hierarchical navigation strategy and a multi-step qualitative evaluation, we can tackle complex questions in long documents with high precision and full traceability, all without the overhead of a traditional vector database.

**Key Takeaways:**

1.  **Agents Offer Control:** Breaking the problem into specialized agents (Router, Synthesizer, Evaluator) provides greater control and allows for the use of different models optimized for each task.
2.  **Hierarchical Navigation is Powerful:** This approach effectively narrows down a vast search space, mimicking human-like research patterns.
3.  **LLM-based Evaluation is Crucial:** Moving beyond simple operational metrics to assess faithfulness and relevance is key to building trustworthy and reliable AI systems.
4.  **Comprehensive Analysis:** Consolidating operational and qualitative metrics into a final summary provides a clear, holistic view of system performance for each query.